In [6]:
import os
import cv2
import joblib
import numpy as np
import mediapipe as mp
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
from scipy.signal import savgol_filter

mp_pose = mp.solutions.pose
pose = mp_pose.Pose()
mp_drawing = mp.solutions.drawing_utils

def calculate_angle(a, b, c):
    a, b, c = np.array(a), np.array(b), np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
    return np.degrees(angle)

def put_text_with_background(image, text, position, font=cv2.FONT_HERSHEY_SIMPLEX, font_scale=0.8, color=(255, 255, 255), thickness=2, bg_color=(0, 0, 0)):
    (text_width, text_height), baseline = cv2.getTextSize(text, font, font_scale, thickness)
    x, y = position
    cv2.rectangle(image, (x, y - text_height - 10), (x + text_width, y + baseline), bg_color, -1)  # Draw background rectangle
    cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)

def process_frame(frame, model, scaler_X, encoder):
    alto_original, ancho_original = frame.shape[:2]
    nuevo_alto = 1100  # Nueva altura deseada
    nuevo_ancho = int((nuevo_alto / alto_original) * ancho_original)
    frame = cv2.resize(frame, (nuevo_ancho, nuevo_alto))

    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        keypoints = [(lm.x, lm.y, lm.z) for lm in landmarks]

        # Extract relevant angles and distances
        left_shoulder = keypoints[11]
        right_shoulder = keypoints[12]
        left_elbow = keypoints[13]
        right_elbow = keypoints[14]
        left_wrist = keypoints[15]
        right_wrist = keypoints[16]
        left_hip = keypoints[23]
        right_hip = keypoints[24]
        left_knee = keypoints[25]
        right_knee = keypoints[26]
        left_ankle = keypoints[27]
        right_ankle = keypoints[28]

        # Example angles
        left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        left_shoulder_angle = calculate_angle(left_hip, left_shoulder, left_elbow)
        right_shoulder_angle = calculate_angle(right_hip, right_shoulder, right_elbow)
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)

        features = np.array(keypoints).flatten().tolist()
        X = scaler_X.transform(np.array(features).reshape(1, -1))
        X = X[:, :, np.newaxis] 

        predictions = model.predict(X)

        predicciones_continuas = predictions[0][0]
        prediccion_clase = encoder.inverse_transform(np.argmax(predictions[1], axis=1))

        features_video = [left_elbow_angle, right_elbow_angle, left_shoulder_angle, right_shoulder_angle, left_knee_angle, right_knee_angle]

        porcentaje_diferencia = 0.25
        caracteristicas = ["Angulo del codo izquierdo", "Angulo del codo derecho", "Angulo del hombro izquierdo", "Angulo del hombro derecho", "Angulo de la rodilla izquierda", "Angulo de la rodilla derecha"]

        array1 = np.array(predicciones_continuas)
        array2 = np.array(features_video)
        relacion = np.abs(1 - (array1 / array2))
        errores = [caracteristicas[i] for i in range(len(relacion)) if porcentaje_diferencia < relacion[i] < (1 - porcentaje_diferencia)]

        annotated_image = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        mp_drawing.draw_landmarks(
            annotated_image,
            results.pose_landmarks,
            mp_pose.POSE_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)
        )

        clase_predicha = prediccion_clase[0]
        porcentaje_prediccion = np.max(predictions[1]) * 100
        texto_clasificacion = f'Clasificacion: {clase_predicha} - Probabilidad: {porcentaje_prediccion:.2f}%'
        put_text_with_background(annotated_image, texto_clasificacion, (50, 220), font_scale=0.8, bg_color=(0, 0, 0))

        if len(errores) > 0:
            for i, error in enumerate(errores):
                desplazamiento_vertical = 270 + (i * 30)
                put_text_with_background(annotated_image, f'Error en : {error}', (50, desplazamiento_vertical), font_scale=0.8, bg_color=(0, 0, 0))

        return annotated_image

    return frame


def analyze_path(file_path, model, scaler_X, encoder):
    if file_path.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
        cap = cv2.VideoCapture(file_path)
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break

            annotated_frame = process_frame(frame, model, scaler_X, encoder)
            cv2.imshow('Frame', cv2.cvtColor(annotated_frame, cv2.COLOR_RGB2BGR))

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    elif file_path.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
        frame = cv2.imread(file_path)
        annotated_frame = process_frame(frame, model, scaler_X, encoder)
        rgb_image = cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('Image', rgb_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("Formato de archivo no soportado")

# Cargar el modelo y las clases
ruta_actual = os.getcwd()
carpeta_mediapipe = os.path.dirname(ruta_actual)
ruta_raiz = os.path.dirname(carpeta_mediapipe)

ruta_dataset = os.path.join(ruta_raiz, 'Data Set')
ruta_archivo = os.path.join(ruta_dataset, 'test_video_3.mp4')  # Cambia esta ruta a la ruta del archivo que desees analizar
ruta_npy = os.path.join(ruta_actual, 'datos', 'classes.npy')
ruta_scaler = os.path.join(ruta_actual, 'datos', 'scaler_X.pkl')
ruta_modelo = os.path.join(ruta_actual, 'model', 'exercise_model.h5')

# Cargar el modelo y scaler
model = load_model(ruta_modelo)
scaler_X = joblib.load(ruta_scaler)

classes = np.load(ruta_npy, allow_pickle=True)
encoder = LabelEncoder()
encoder.classes_ = classes

# Realizar el análisis en la ruta dada
analyze_path(ruta_archivo, model, scaler_X, encoder)


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step


c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


...

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step

c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
c:\Users\Diego Castillo\anaconda3\envs\Unir_TFM\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
